# Rule Based Classification
University of Denver

Makarand Nadendla

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
import seaborn as sns 

In [2]:
income_tax_df = pd.read_csv("cheat.csv", sep="	")

In [8]:
def predict(tax_row):
    if tax_row["refund"] == "yes":
        return "no"
    else:
        if tax_row["income_above_80k"] == "no":
            return "no"
        else:
            if tax_row["marital_status"] == "married":
                return "no"
            else:
                return "yes"

In [9]:
income_tax_df["predicted_cheat"] = income_tax_df.apply(predict, axis = 1)
income_tax_df.head(10)

,refund,marital_status,income_above_80k,cheat,predicted_cheat
0,yes,single,yes,no,no
1,no,married,yes,no,no
2,no,single,no,no,no
3,yes,married,yes,no,no
4,no,divorced,yes,yes,yes
5,no,married,no,no,no
6,yes,divorced,yes,no,no
7,no,single,yes,yes,yes
8,no,married,no,no,no
9,no,single,yes,yes,yes


In [31]:
def vector_predict(tax_df):
    cheat_predict_vector = np.where((income_tax_df["refund"] == "no") & (income_tax_df["income_above_80k"] == "yes") & ((income_tax_df["marital_status"] == "single") | (income_tax_df["marital_status"] == "divorced")), 1, 0)
    return cheat_predict_vector

In [32]:
vector_predict(income_tax_df)

array([0, 0, 0, 0, 1, 0, 0, 1, 0, 1])

In [90]:
def overall_accuracy(predict, ground_truth):
    total = len(predict)
    matching_entries = 0
    for i, v in zip(predict, ground_truth):
        if i == v:
            matching_entries += 1
    return matching_entries/total

In [10]:
overall_accuracy(income_tax_df["predicted_cheat"], income_tax_df["cheat"])

1.0

In [11]:
pd.crosstab(income_tax_df["predicted_cheat"], income_tax_df["cheat"])

cheat,no,yes
predicted_cheat,,
no,7,0
yes,0,3


In [87]:
income_tax_df_2 = pd.read_csv("cheat2.csv", sep="	")
income_tax_df_2.head(10)

,refund,marital_status,income_above_80k,cheat
0,no,single,yes,no
1,no,single,yes,no
2,no,married,yes,no
3,no,divorced,no,no
4,no,married,yes,no
5,no,single,yes,no
6,yes,single,yes,no
7,no,single,yes,yes
8,yes,married,yes,yes
9,yes,single,no,yes


In [18]:
income_tax_df_2["predicted_cheat"] = income_tax_df_2.apply(predict, axis = 1)
income_tax_df_2.head(10)

,refund,marital_status,income_above_80k,cheat,predicted_cheat
0,no,single,yes,no,yes
1,no,single,yes,no,yes
2,no,married,yes,no,no
3,no,divorced,no,no,no
4,no,married,yes,no,no
5,no,single,yes,no,yes
6,yes,single,yes,no,no
7,no,single,yes,yes,yes
8,yes,married,yes,yes,no
9,yes,single,no,yes,no


In [19]:
overall_accuracy(income_tax_df_2["cheat"] ,income_tax_df_2["predicted_cheat"] )

0.5

In [32]:
def entropy(matrix, target):
    vector = matrix[target]
 
    prop = vector.value_counts()/len(vector)
    total_entropy = 0

    # calculate the entropy
    for p in prop:
        total_entropy += p*np.log2(p)

    return -total_entropy

In [33]:
entropy(income_tax_df, "cheat")

0.8812908992306927

In [18]:
def vector_entropy(vector):

    prop = vector.value_counts()/len(vector)
    total_entropy = 0

    # calculate the entropy
    for p in prop:
        total_entropy += p*np.log2(p)

    return -total_entropy

In [20]:
def information_gain(matrix, target, split):
    
    length = matrix.shape[0]

    entropy_sum = 0
    # split the target variable on the split variable
    for cheat_split in matrix.groupby(split):

        split_vector = cheat_split[1][target]
    # calculate the length of each split
        split_len = len(split_vector)

    # calculate the entropy of each split, multiply it by the split proportion, and then sum them all together
        entropy_sum += (split_len/length)*vector_entropy(split_vector)
    
    entropy_sum = vector_entropy(matrix[target]) - entropy_sum

    return entropy_sum

In [26]:
information_gain(income_tax_df, "marital_status", "cheat")

0.2812908992306926

In [40]:
pd.crosstab(income_tax_df["marital_status"], income_tax_df["cheat"])

cheat,no,yes
marital_status,,
divorced,1,1
married,4,0
single,2,2


In [41]:
def coverage(matrix, cover_cond):
    length = matrix.shape[0]
    cover_len = matrix[cover_cond].shape[0]

    coverage = cover_len/length
    return coverage

In [43]:
coverage(income_tax_df, income_tax_df["marital_status"]=="married")

0.4

In [84]:
def one_rule(row):
    if row["marital_status"] == "married":
        return "no"
    else:
        return "yes"

def predict_one_rule(matrix):
    if len(matrix.shape) > 1:
        cheat_predict = matrix.apply(one_rule, axis = 1)
    else:
        cheat_predict = np.where(matrix == "married", "no", "yes")
    return cheat_predict
       

In [73]:
def accuracy(matrix, cover_cond):
    length = matrix.shape[0]

    match_len = sum(matrix[cover_cond]["cheat"] == "no")

    accuracy = match_len/length
    return accuracy

In [74]:
accuracy(income_tax_df, income_tax_df["marital_status"]=="married")

0.4

In [91]:
income_tax_df["predicted_cheat_one_rule"] = predict_one_rule(income_tax_df)

In [92]:
income_tax_df_2["predicted_cheat_one_rule"] = predict_one_rule(income_tax_df_2)

In [94]:
overall_accuracy(income_tax_df["predicted_cheat_one_rule"], income_tax_df["cheat"])

0.7

In [95]:
overall_accuracy(income_tax_df_2["predicted_cheat_one_rule"], income_tax_df_2["cheat"])

0.4